In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\dataDownloaded'

In [4]:
img_folder = r'D:\original'
out_folder = r'D:\ToAWS2'

course_list = ['청도','포항','중문','시흥','거창','속리산','춘천']

course_ids = {
  '포항':'MGC001',
  '청도':'MGC002',
  '중문':'MGC003',
  '시흥':'MGC004',
  '거창':'MGC005',
  '속리산':'MGC006',
  '춘천':'MGC007'
}

folder_type = ['rgb','ndvi','thumb']

In [5]:

course_id = course_ids.get('청도')
course_id

'MGC002'

In [6]:
folders_candidate =glob.glob(os.path.join(out_folder,course_id,'*'))
folders_candidate

['D:\\ToAWS2\\MGC002\\20230823',
 'D:\\ToAWS2\\MGC002\\20230907',
 'D:\\ToAWS2\\MGC002\\20230919',
 'D:\\ToAWS2\\MGC002\\20230927']

In [7]:
target_folder = folders_candidate[0]
target_folder

'D:\\ToAWS2\\MGC002\\20230823'

In [8]:
target_date = os.path.split(target_folder)[-1]
target_date 

'20230823'

Gather all data json files from Output Folder

In [9]:
dataJsonList = glob.glob(os.path.join(out_folder,course_id,target_date,'**/data*.json'), recursive=True)
dataJsonList

['D:\\ToAWS2\\MGC002\\20230823\\10_스타트퍼팅그린\\data2023082310_스타트퍼팅그린.json',
 'D:\\ToAWS2\\MGC002\\20230823\\1_밸리6H\\data202308231_밸리6H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\2_밸리7H\\data202308232_밸리7H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\3_마운틴너스리포장\\data202308233_마운틴너스리포장.json',
 'D:\\ToAWS2\\MGC002\\20230823\\4_마운틴1H\\data202308234_마운틴1H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\5_마운틴2H\\data202308235_마운틴2H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\6_마운틴3H\\data202308236_마운틴3H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\7_레이크2H\\data202308237_레이크2H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\8_레이크3H\\data202308238_레이크3H.json',
 'D:\\ToAWS2\\MGC002\\20230823\\9_레이크4H\\data202308239_레이크4H.json']

In [10]:
folderinDataJson = []
totalPhotoJson = []

for dataJson_ in dataJsonList:
  with open(dataJson_, "r", encoding='utf-8') as _json:
    dataJson= json.load(_json)


  for data_ in dataJson:
    # if data_['dest']['thumb'][0] !="M" :
      data_['dest']['thumb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'thumb','thumbrgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['rgb'][0] !="M" :
      data_['dest']['rgb'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'rgb','rgb{}.JPG'.format( data_['id']) )
    # if data_['dest']['ndvi'][0] !="M" :
      data_['dest']['ndvi'] = os.path.join(course_id,target_date,data_['info']['area'],data_['info']['desc'],'ndvi','ndvi{}.JPG'.format( data_['id']) )
    
  
  totalPhotoJson.extend(dataJson)

  for Json_ in dataJson:
    if Json_['destFolder'] not in folderinDataJson:
      folderinDataJson.append(Json_['destFolder'])




In [11]:
len(totalPhotoJson)

8319

In [16]:
oldLabelList = glob.glob(os.path.join(b_folder,'*.json'), recursive=True)
oldLabelList = [x for x in oldLabelList if course_id in x and target_date  in x]
oldLabelList 

['c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\BACKUPS\\MGC002_20230823.json']

In [18]:
with open(oldLabelList[0], "r", encoding='utf-8') as _json:
  oldlabel = json.load(_json)

In [40]:
old_df = pd.DataFrame(oldlabel)
old_df.shape

(8320, 7)

In [41]:
old_df.head(2)

,metadata,date,area,courseid,json,id,desc
0,"MGC002202308231_밸리6H건조,패취류,썸머패취,켄터키,티,답압,장비피해",20230823,1_밸리6H,MGC002,{'originalFileGrp': ['original\2023_0823_청도그레이...,20230823103730_1286504149444_356546272778_1306...,"건조,패취류,썸머패취,켄터키,티,답압,장비피해"
1,"MGC002202308231_밸리6H건조,패취류,썸머패취,켄터키,티,답압,장비피해",20230823,1_밸리6H,MGC002,{'originalFileGrp': ['original\2023_0823_청도그레이...,20230823103732_1286504147222_356546285833_1306...,"건조,패취류,썸머패취,켄터키,티,답압,장비피해"


In [42]:
old_df['newlabel'] = old_df.json.map(lambda x: x['label'])
old_df = old_df[['id', 'newlabel']]

In [43]:
df = pd.DataFrame(totalPhotoJson)
df.shape



(8319, 10)

In [44]:
df= df.merge(old_df, on = ['id'], how = 'left')
df.shape

(8319, 11)

In [45]:
df.head(2)

,id,originalFileJPG,originalFileGrp,destFolder,GeoTagInfo,dest,info,label,annotation,labelBy,newlabel
0,20230823162119_1286447212222_356634586944_1599...,NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도그...,[NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도...,"20230823/10_스타트퍼팅그린/엽고성병,달라스팟,벤트,그린_0","{'coords': [128.64472122222222, 35.66345869444...","{'rgb': 'MGC002\20230823\10_스타트퍼팅그린\엽고성병,달라스팟,...","{'course': '청도', 'area': '10_스타트퍼팅그린', 'desc':...",[],[],,"[{'level1': '물리피해', 'TurfType': '벤트그래스', 'leve..."
1,20230823162121_1286447209722_356634581111_1599...,NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도그...,[NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도...,"20230823/10_스타트퍼팅그린/엽고성병,달라스팟,벤트,그린_0","{'coords': [128.64472097222222, 35.66345811111...","{'rgb': 'MGC002\20230823\10_스타트퍼팅그린\엽고성병,달라스팟,...","{'course': '청도', 'area': '10_스타트퍼팅그린', 'desc':...",[],[],,"[{'level1': '물리피해', 'TurfType': '벤트그래스', 'leve..."


In [46]:
df.label = df.newlabel
df = df[df.columns[:-1]]
df.head(2)

,id,originalFileJPG,originalFileGrp,destFolder,GeoTagInfo,dest,info,label,annotation,labelBy
0,20230823162119_1286447212222_356634586944_1599...,NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도그...,[NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도...,"20230823/10_스타트퍼팅그린/엽고성병,달라스팟,벤트,그린_0","{'coords': [128.64472122222222, 35.66345869444...","{'rgb': 'MGC002\20230823\10_스타트퍼팅그린\엽고성병,달라스팟,...","{'course': '청도', 'area': '10_스타트퍼팅그린', 'desc':...","[{'level1': '물리피해', 'TurfType': '벤트그래스', 'leve...",[],
1,20230823162121_1286447209722_356634581111_1599...,NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도그...,[NIA_2023_52_잔디\10_원시데이터(3사_분류대상)\2023_0823_청도...,"20230823/10_스타트퍼팅그린/엽고성병,달라스팟,벤트,그린_0","{'coords': [128.64472097222222, 35.66345811111...","{'rgb': 'MGC002\20230823\10_스타트퍼팅그린\엽고성병,달라스팟,...","{'course': '청도', 'area': '10_스타트퍼팅그린', 'desc':...","[{'level1': '물리피해', 'TurfType': '벤트그래스', 'leve...",[],


In [47]:
save_name = os.path.join(out_folder,course_id,target_date, 'photo.json')
with open(save_name, "w", encoding='utf-8') as f:
   f.write(df.to_json(orient='records', force_ascii=False))

In [48]:
filestructure = []


for folder_ in folderinDataJson:
  fileinfo = {}
  fileinfo["path"] = folder_
  fileinfo["file_cnt"] = len([ x for x in totalPhotoJson if x["destFolder"] == folder_])
  fileinfo["labeled_file_cnt"] = 0
  filestructure.append(fileinfo)
  

In [49]:
save_name = os.path.join(out_folder,course_id,target_date, 'filepath_'+course_id + '_'+target_date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(filestructure, final , ensure_ascii=False)